# Working Version From Machinelearning mastery

In [1]:
# SG Sigmoid in Generator 
# SL Soft Labels
# GN Gaussian Noise (not uniform)
# OHE One Hot Encoding
# Replaced Concatenate with multiply

# TODO
# basic conditional gan
# generate data
# add and train a classifier use k fold also
# condition me one hot encoding
# generator k lie combined data and disc k lie train data.. coz testing me sara deke bias ho jaega
# different loss functions for generator and discriminator
# discriminator output shud be one hot encode with labels as n s or v and not 0 or 1 that is fake or real:
# confusion here
# disc property: should recognise fake and also if gen data is not of that class (so 2 things)
# training k time jab random samples uthaenge tab sari classes k
# equal no. of samples uthana

In [2]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

In [3]:
def discriminator(data_dim, beat_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    D_in = Input(shape=[data_dim,1])
    x = Concatenate()([D_in, in_label])
    
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*16, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[D_in, in_label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model

# d_model = discriminator(data_dim=186, beat_dim=186)
# plot_model(d_model, to_file='disc.png', show_shapes=True)

In [4]:
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf

def generator(noise_dim=186, beat_dim=186, out_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    G_in = Input(shape=[noise_dim,1])
    x = Concatenate()([G_in, in_label])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=2, strides=2, padding='valid', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('sigmoid')(x)  # for tanh change here
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=186, beat_dim=186, out_dim=186)
# plot_model(g_model, to_file='gen.png', show_shapes=True)

In [5]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_beat = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_beat])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_beat], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='gan.png', show_shapes=True)

In [6]:
def load_real_samples():
    X = np.load('Data/X.npy')
    y = np.load('Data/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)

    X_N=X_N.reshape(X_N.shape[0],X_N.shape[1],1)
    X_S=X_S.reshape(X_S.shape[0],X_S.shape[1],1)
    X_V=X_V.reshape(X_V.shape[0],X_V.shape[1],1)

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)
    return X_N, y_N, X_S, y_S, X_V, y_V

In [7]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
#     labels = np.hstack((y_N[i_N], y_S[i_S], y_V[i_V]))
#     labels = keras.utils.to_categorical(labels)
    # print (labels.shape)
    
    # generate class labels
    y = np.random.uniform(0.7, 1, n_samples)
    y = y.reshape(y.shape[0], 1)
#     y = np.ones((n_samples, 1))
#     return [X, labels], y
    return X, y

In [8]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    return reshape(X_fake)

In [9]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, z_input])
    # create class labels
    y = np.random.uniform(0, 0.3, n_samples)
    y = y.reshape(y.shape[0], 1)
#     y = np.zeros((n_samples, 1))
    return images, y

In [10]:
# create and save a plot of generated images
def save_plot(X, n):
    plt.figure(figsize=(10,3))
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.show()
    plt.close()

In [11]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

def save_new_plot(X, n, name):
    plt.figure(figsize=(15,5))
    for i in range(n-1):
        plt.subplot(n+1, n-1, 1 + i)
        plt.axis('off')
        plt.plot(X[i, :, 0])
    for i in range(3, (n*n)-n):
        # define subplot
        plt.subplot(n+1, n-1, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.savefig(name, dpi=50)
    plt.close()

In [ ]:
# size of the latent space
latent_dim = 186
# size of the data
data = 186
# classes
classes = 3

n_epochs=10

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# samples to watch
n=3

# Loss Values
D_L_1 = np.infty
D_L_2 = np.infty
G_L = np.infty

# create the discriminator
d_model = discriminator(data_dim=data)
# d_model = discriminator(data_dim=data, input_classes=classes)

# create the generator
g_model = generator(noise_dim=data, beat_dim=data, out_dim=data)
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)

# create the gan
gan_model = create_gan(d_model, g_model)

# load ECG data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
temp_N = np.mean(X_N, axis=0)
temp_S = np.mean(X_S, axis=0)
temp_V = np.mean(X_V, axis=0)

# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)
flag=0

N = np.tile(temp_N,(1,int(half_batch/3)))
S = np.tile(temp_S,(1,int(half_batch/3)))
V = np.tile(temp_V,(1,int(half_batch/3)))
final_labels = reshape(np.hstack((N,S,V)).T)

final_labels1 = np.vstack((final_labels, final_labels))
# print (final_labels.shape, final_labels1.shape)

# losses = []
plt.ioff()

filename = 'Beat_Morphology_Label'
# filename = 'Multiply_Tanh_OHE'

if not os.path.isdir(filename):
    os.mkdir(filename)

f = open(filename + '_Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, final_labels.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch([X_real, final_labels], y_real)
        
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch([X_fake, X_fake], y_fake)
        
        # prepare points in latent space as input for the generator
        z_input = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = reshape(np.random.uniform(0.7, 1, n_batch))
        # print (z_input.shape, y_gan.shape)
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch([z_input, final_labels1], y_gan)
        
        if (d_loss1 < D_L_1 and d_loss2 < D_L_2) or g_loss < G_L:
            D_L_1 = d_loss1
            D_L_2 = d_loss2
            G_L = g_loss
            g_model.save(filename + '_cgan_generator.h5')
            
            
#         losses.append((d_loss1, d_loss2, g_loss))
        f = open(filename + '_Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()
        
        # summarize loss on this batch
        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
#         if (j+1)%5 == 0:
            name = filename + '/'+str(i*1000 + j)+'.jpg'
            # generate images
            latent_points = generate_latent_points(latent_dim, n*n)
            # specify labels
            X, y = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n*n)
            # generate images
            X  = g_model.predict([latent_points, X])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            X = np.vstack((X_R, X))
            save_new_plot(X, n+1, name)

>1, 2/80, d1=0.48109, d2=0.66606 g=1.28314
>1, 4/80, d1=0.44442, d2=0.52309 g=1.18656
>1, 6/80, d1=0.43232, d2=0.46297 g=1.31544
>1, 8/80, d1=0.44797, d2=0.46930 g=1.32830
>1, 10/80, d1=0.43315, d2=0.46864 g=1.24781
>1, 12/80, d1=0.44487, d2=0.44316 g=1.19401
>1, 14/80, d1=0.44520, d2=0.45552 g=1.19821
>1, 16/80, d1=0.45884, d2=0.44868 g=1.02776
>1, 18/80, d1=0.44723, d2=0.45409 g=0.99277
>1, 20/80, d1=0.43148, d2=0.46593 g=0.94243
>1, 22/80, d1=0.43716, d2=0.46099 g=0.85718
>1, 24/80, d1=0.40837, d2=0.45382 g=0.78570
>1, 26/80, d1=0.42844, d2=0.46323 g=0.77018
>1, 28/80, d1=0.42185, d2=0.44222 g=0.82962
>1, 30/80, d1=0.44368, d2=0.43345 g=0.78568
>1, 32/80, d1=0.44797, d2=0.44712 g=0.73341
>1, 34/80, d1=0.46575, d2=0.46406 g=0.84015
>1, 36/80, d1=0.46485, d2=0.42896 g=0.92891
>1, 38/80, d1=0.45303, d2=0.43499 g=0.92315
>1, 40/80, d1=0.45378, d2=0.43537 g=0.84038
>1, 42/80, d1=0.44004, d2=0.42643 g=0.93808
>1, 44/80, d1=0.46118, d2=0.42529 g=0.97209
>1, 46/80, d1=0.44137, d2=0.42927 g=

>5, 56/80, d1=0.45009, d2=0.43628 g=0.43223
>5, 58/80, d1=0.44233, d2=0.43380 g=0.44757
>5, 60/80, d1=0.43130, d2=0.43965 g=0.42732
>5, 62/80, d1=0.43686, d2=0.43805 g=0.41004
>5, 64/80, d1=0.41007, d2=0.43338 g=0.43518
>5, 66/80, d1=0.44726, d2=0.42789 g=0.41891
>5, 68/80, d1=0.47809, d2=0.43650 g=0.43919
>5, 70/80, d1=0.41521, d2=0.44145 g=0.42709
>5, 72/80, d1=0.41980, d2=0.39872 g=0.42773
>5, 74/80, d1=0.40902, d2=0.43602 g=0.40938
>5, 76/80, d1=0.41327, d2=0.46692 g=0.43778
>5, 78/80, d1=0.43549, d2=0.42662 g=0.42480
>5, 80/80, d1=0.41634, d2=0.43227 g=0.40849
>6, 2/80, d1=0.41522, d2=0.43112 g=0.41583
>6, 4/80, d1=0.41942, d2=0.43481 g=0.42664
>6, 6/80, d1=0.41146, d2=0.43660 g=0.45652
>6, 8/80, d1=0.43477, d2=0.43040 g=0.43404
>6, 10/80, d1=0.41431, d2=0.40867 g=0.43038
>6, 12/80, d1=0.42979, d2=0.44422 g=0.43775
>6, 14/80, d1=0.42921, d2=0.41643 g=0.42197
>6, 16/80, d1=0.42006, d2=0.42360 g=0.43918
>6, 18/80, d1=0.43745, d2=0.45084 g=0.42845
>6, 20/80, d1=0.43062, d2=0.43690 g=

In [ ]:
# from evaluation_metrics import *
# metric_to_calculate = ['FID', 'MMD', 'DTW', 'ED', 'PC', 'KLD', 'RMSE', 'TWED']
# f = open(data_dir+'Stats.csv', 'a')
# f.write('Epoch, d_loss1, d_loss2, g_loss, ')
# for i in range(len(label_dict)):
#     for mtc in metric_to_calculate:
#         f.write(str(mtc)+str(i)+',')
# f.write('Time,\n')
# f.close()

# d_loss1 = float(d_loss1.history['loss'][0])
# d_loss2 = float(d_loss2.history['loss'][0])
# g_loss = float(g_loss.history['loss'][0])

# if i%checkpoint == 0:
        
#         f = open(data_dir+'Stats.csv', 'a')
#         f.write(str(i+1)+','+str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+',')
#         save_model(model=G, data_dir=data_dir, type='G', epoch=i)
#         save_model(model=D, data_dir=data_dir, type='D', epoch=i)
#         save_model(model=GAN, data_dir=data_dir, type='GAN', epoch=i)
        
#         for k,metric in enumerate(label_dict.keys()):    
#             temp_x = test_data[200*(k):200*(k+1),:-1]
#             [z_input, labels_input] = generate_class_specific_latent_input(200, n_classes=n_classes, noise_dim=noise_dim, category=float(metric))
#             z_input = G.predict([z_input, labels_input], verbose=verbose)

#             for j in range(2):
#                 plt.plot(z_input[j])
#             plt.savefig(data_dir+str(i)+'_Label_'+str(metric)+'.png')
#             plt.close()
#             plt.clf()
            
#             results = evaluate(temp_x,z_input,metric_to_calculate)
#             for r in results:
#                 f.write(str(r)+',')
            
#         f.write(str(end-start)+'\n')
#         f.close()

In [ ]:
# #     # save the generator model
#     g_model.save('cgan_generator.h5')

# callback = [EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=Pat),
#          ModelCheckpoint(filepath=str(twelve_lead_model_filename)+'_check_model.h5', 
#                          monitor='val_AUC', verbose=1, save_best_only=True, mode='max'),
#          ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=Pat//2, verbose=1, 
#                            mode='max', min_delta=0.0001, cooldown=0, min_lr=0)]

# # model = parallel_NN(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
# model = parallel_NN(Window, len(leads), snomed_classes.shape[0])

# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
#               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
#               metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),
#                        tf.keras.metrics.Recall(name='Recall'),
#                        tf.keras.metrics.Precision(name='Precision'),
#                        tf.keras.metrics.AUC(num_thresholds=200,summation_method="interpolation",
#                                             name="AUC",dtype=None,curve="ROC",thresholds=None,
#                                             multi_label=True,label_weights=None)])
# history = model.fit(train_generator, steps_per_epoch=train_samples, epochs=EP, verbose=1,
#                 validation_data=val_generator, validation_steps=val_samples, callbacks=callback)


# history_name = output_directory + '/' + twelve_lead_filename
# print (twelve_lead_model_filename, history_name)

# save_model(twelve_lead_model_filename, model)
# write_history(history_name, history.history)